In [0]:
import pandas as pd
import numpy as np
import torch
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms, datasets


In [196]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [222]:
df = pd.read_csv('/content/gdrive/My Drive/ECG_data/test.csv')
df.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,65497,65498,65499,65500,65501,65502,65503,65504,65505,65506,65507,65508,65509,65510,65511,65512,65513,65514,65515,65516,65517,65518,65519,65520,65521,65522,65523,65524,65525,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535,Label
0,0,-0.097857,-0.156881,-0.137816,-0.137177,-0.147684,-0.168522,-0.175891,-0.170236,-0.181131,-0.146471,-0.212436,-0.239700,-0.271648,-0.266576,-0.268720,-0.290667,-0.291689,-0.290488,-0.276202,-0.298062,-0.287771,-0.310729,-0.355139,-0.408453,-0.481971,-0.307582,0.160449,0.782387,0.580402,-0.285574,-0.405491,-0.324070,-0.337285,-0.311032,-0.335907,-0.334622,-0.332370,-0.328278,-0.347875,...,-0.194847,-0.207336,-0.199616,-0.207660,-0.199723,-0.213992,-0.210273,-0.216094,-0.218033,-0.223974,-0.231275,-0.239940,-0.242455,-0.242408,-0.257434,-0.242908,-0.247033,-0.232162,-0.240069,-0.225888,-0.244432,-0.237713,-0.243042,-0.232664,-0.232344,-0.221831,-0.191805,-0.176982,-0.161231,-0.161495,-0.124765,-0.105196,-0.094394,-0.124967,-0.104613,-0.126682,-0.074905,-0.116527,-0.180421,"[array(['ARR'], dtype='<U3')]"
1,1,0.119361,0.187559,0.169569,0.178590,0.211824,0.196607,0.072911,0.165885,0.083204,0.199233,0.178986,0.111600,0.120170,0.114239,0.023420,-0.012190,-0.095595,-0.118857,-0.115468,-0.144367,-0.156838,-0.181808,-0.174113,-0.197013,-0.193959,-0.204466,-0.205493,-0.168962,-0.189202,-0.169633,-0.182248,-0.189404,-0.185059,-0.236682,-0.128320,0.147378,0.211654,0.436271,0.626556,...,-0.780243,-0.693552,-0.652115,-0.591565,-0.526449,-0.535017,-0.521135,-0.539240,-0.527915,-0.557910,-0.551029,-0.557357,-0.554629,-0.558668,-0.555358,-0.554652,-0.560797,-0.536543,-0.530443,-0.488854,-0.474442,-0.416927,-0.398104,-0.349849,-0.315034,-0.264330,-0.235463,-0.188696,-0.161925,-0.141731,-0.121423,-0.113781,-0.087638,-0.097274,-0.089068,-0.105625,-0.097218,-0.123936,-0.106481,"[array(['ARR'], dtype='<U3')]"
2,2,-0.024602,-0.036279,-0.035968,-0.030194,-0.045812,-0.093124,-0.091381,-0.128953,-0.112480,-0.127330,-0.105625,-0.124254,-0.119365,-0.121182,-0.113506,-0.108732,-0.120119,-0.111383,-0.110269,-0.091718,-0.122786,-0.107965,-0.134320,-0.125777,-0.143800,-0.131847,-0.145230,-0.124747,-0.139210,-0.128165,-0.141062,-0.144168,-0.144695,-0.155978,-0.141107,-0.145403,-0.118500,-0.117982,-0.079222,...,-0.361945,-0.332084,-0.308275,-0.234087,-0.139343,0.045783,0.066538,-0.123800,-0.260275,0.019546,0.358345,0.487138,0.232889,-0.040127,-0.355037,-0.560554,-0.616312,-0.581124,-0.453867,-0.418127,-0.399276,-0.415678,-0.380331,-0.389416,-0.371190,-0.379303,-0.362888,-0.358385,-0.338570,-0.327886,-0.308444,-0.282376,-0.269384,-0.248776,-0.247613,-0.191715,-0.186142,-0.129985,-0.122356,"[array(['ARR'], dtype='<U3')]"
3,3,-0.508309,-0.808009,-0.721684,-0.789720,-0.767678,-0.774352,-0.772501,-0.794323,-0.794939,-0.778141,-0.786919,-0.811797,-0.808065,-0.791179,-0.778835,-0.795678,-0.796654,-0.776672,-0.747686,-0.722190,-0.713651,-0.709948,-0.679208,-0.670205,-0.672490,-0.707004,-0.734000,-0.748158,-0.756905,-0.811784,-0.829412,-0.837680,-0.859322,-0.866709,-0.862564,-0.848966,-0.849785,-0.862873,-0.861149,...,-0.771301,-0.788602,-0.792420,-0.773845,-0.752200,-0.766477,-0.713998,-0.495185,-0.128602,0.141811,-0.031988,-0.453074,-0.876338,-1.059069,-0.988916,-0.916877,-0.897284,-0.892269,-0.896820,-0.905748,-0.886943,-0.886123,-0.891974,-0.918370,-0.923204,-0.908035,-0.915812,-0.933878,-0.951456,-0.953446,-0.943424,-0.971548,-0.988559,-0.994919,-0.996746,-1.015446,-1.042696,-1.040684,-1.048192,"[array(['ARR'], dtype='<U3')]"
4,4,-0.312039,-0.495260,-0.442597,-0.465246,-0.428401,-0.444373,-0.417601,-0.453342,-0.429223,-0.441911,-0.422556,-0.443607,-0.431997,-0.430010,-0.423942,-0.428989,-0.429649,-0.412002,-0.432764,-0.415027,-0.434078,-0.409519,-0.436252,-0.408323,-0.432162,-0.411869,-0.423802,-0.414550,-0.414504,-0.425122,-0.400945,-0.409350,-0.404152,-0.412800,-0.391469,-0.403646,-0.389578,-0.397415,-0

In [223]:
#df.columns()
df = df.drop(columns = ['Unnamed: 0'])
#df4 = df.rename({'Unnamed: 0': 'id'}, axis=1)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,65497,65498,65499,65500,65501,65502,65503,65504,65505,65506,65507,65508,65509,65510,65511,65512,65513,65514,65515,65516,65517,65518,65519,65520,65521,65522,65523,65524,65525,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535,Label
0,-0.097857,-0.156881,-0.137816,-0.137177,-0.147684,-0.168522,-0.175891,-0.170236,-0.181131,-0.146471,-0.212436,-0.239700,-0.271648,-0.266576,-0.268720,-0.290667,-0.291689,-0.290488,-0.276202,-0.298062,-0.287771,-0.310729,-0.355139,-0.408453,-0.481971,-0.307582,0.160449,0.782387,0.580402,-0.285574,-0.405491,-0.324070,-0.337285,-0.311032,-0.335907,-0.334622,-0.332370,-0.328278,-0.347875,-0.315589,...,-0.194847,-0.207336,-0.199616,-0.207660,-0.199723,-0.213992,-0.210273,-0.216094,-0.218033,-0.223974,-0.231275,-0.239940,-0.242455,-0.242408,-0.257434,-0.242908,-0.247033,-0.232162,-0.240069,-0.225888,-0.244432,-0.237713,-0.243042,-0.232664,-0.232344,-0.221831,-0.191805,-0.176982,-0.161231,-0.161495,-0.124765,-0.105196,-0.094394,-0.124967,-0.104613,-0.126682,-0.074905,-0.116527,-0.180421,"[array(['ARR'], dtype='<U3')]"
1,0.119361,0.187559,0.169569,0.178590,0.211824,0.196607,0.072911,0.165885,0.083204,0.199233,0.178986,0.111600,0.120170,0.114239,0.023420,-0.012190,-0.095595,-0.118857,-0.115468,-0.144367,-0.156838,-0.181808,-0.174113,-0.197013,-0.193959,-0.204466,-0.205493,-0.168962,-0.189202,-0.169633,-0.182248,-0.189404,-0.185059,-0.236682,-0.128320,0.147378,0.211654,0.436271,0.626556,0.558110,...,-0.780243,-0.693552,-0.652115,-0.591565,-0.526449,-0.535017,-0.521135,-0.539240,-0.527915,-0.557910,-0.551029,-0.557357,-0.554629,-0.558668,-0.555358,-0.554652,-0.560797,-0.536543,-0.530443,-0.488854,-0.474442,-0.416927,-0.398104,-0.349849,-0.315034,-0.264330,-0.235463,-0.188696,-0.161925,-0.141731,-0.121423,-0.113781,-0.087638,-0.097274,-0.089068,-0.105625,-0.097218,-0.123936,-0.106481,"[array(['ARR'], dtype='<U3')]"
2,-0.024602,-0.036279,-0.035968,-0.030194,-0.045812,-0.093124,-0.091381,-0.128953,-0.112480,-0.127330,-0.105625,-0.124254,-0.119365,-0.121182,-0.113506,-0.108732,-0.120119,-0.111383,-0.110269,-0.091718,-0.122786,-0.107965,-0.134320,-0.125777,-0.143800,-0.131847,-0.145230,-0.124747,-0.139210,-0.128165,-0.141062,-0.144168,-0.144695,-0.155978,-0.141107,-0.145403,-0.118500,-0.117982,-0.079222,-0.085295,...,-0.361945,-0.332084,-0.308275,-0.234087,-0.139343,0.045783,0.066538,-0.123800,-0.260275,0.019546,0.358345,0.487138,0.232889,-0.040127,-0.355037,-0.560554,-0.616312,-0.581124,-0.453867,-0.418127,-0.399276,-0.415678,-0.380331,-0.389416,-0.371190,-0.379303,-0.362888,-0.358385,-0.338570,-0.327886,-0.308444,-0.282376,-0.269384,-0.248776,-0.247613,-0.191715,-0.186142,-0.129985,-0.122356,"[array(['ARR'], dtype='<U3')]"
3,-0.508309,-0.808009,-0.721684,-0.789720,-0.767678,-0.774352,-0.772501,-0.794323,-0.794939,-0.778141,-0.786919,-0.811797,-0.808065,-0.791179,-0.778835,-0.795678,-0.796654,-0.776672,-0.747686,-0.722190,-0.713651,-0.709948,-0.679208,-0.670205,-0.672490,-0.707004,-0.734000,-0.748158,-0.756905,-0.811784,-0.829412,-0.837680,-0.859322,-0.866709,-0.862564,-0.848966,-0.849785,-0.862873,-0.861149,-0.849549,...,-0.771301,-0.788602,-0.792420,-0.773845,-0.752200,-0.766477,-0.713998,-0.495185,-0.128602,0.141811,-0.031988,-0.453074,-0.876338,-1.059069,-0.988916,-0.916877,-0.897284,-0.892269,-0.896820,-0.905748,-0.886943,-0.886123,-0.891974,-0.918370,-0.923204,-0.908035,-0.915812,-0.933878,-0.951456,-0.953446,-0.943424,-0.971548,-0.988559,-0.994919,-0.996746,-1.015446,-1.042696,-1.040684,-1.048192,"[array(['ARR'], dtype='<U3')]"
4,-0.312039,-0.495260,-0.442597,-0.465246,-0.428401,-0.444373,-0.417601,-0.453342,-0.429223,-0.441911,-0.422556,-0.443607,-0.431997,-0.430010,-0.423942,-0.428989,-0.429649,-0.412002,-0.432764,-0.415027,-0.434078,-0.409519,-0.436252,-0.408323,-0.432162,-0.411869,-0.423802,-0.414550,-0.414504,-0.425122,-0.400945,-0.409350,-0.404152,-0.412800,-0.391469,-0.403646,-

In [0]:
df_list = []
for i in range(5):
  df1 = pd.DataFrame(data = df.iloc[:,i*1000:(i+1)*1000])
  df1.columns = np.arange(1000)
  df1['Label'] = df['Label']
  df_list.append(df1)
df2 = pd.concat(df_list)
df2.to_csv('test_1000.csv')

In [244]:
df10 = pd.read_csv('test_1000.csv')
df10['Label'][0][9:12]

'ARR'

In [0]:
class ECG_dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.ecg_frame = pd.read_csv(csv_file)
        self.ecg_frame = self.ecg_frame.drop(columns = ['Unnamed: 0'])
        self.transform = transform

    def __len__(self):
        return len(self.ecg_frame)
    
    def __getitem__(self, idx):
      if torch.is_tensor(idx):
        idx = idx.tolist()
      signal = np.array(self.ecg_frame.iloc[idx, :-1])
      label = self.ecg_frame.iloc[idx,-1][9:12]
      label = label
      sample = {'signal': signal, 'label': label}
      if self.transform:
          sample = self.transform(sample)
      return sample


# class ToTensor(object):
#     """Convert ndarrays in sample to Tensors."""

#     def __call__(self, sample):
#         signal, label = sample['signal'], sample['label']

#         return {'signal': torch.from_numpy(signal),
#                 'label': torch.from_numpy(label)}

In [0]:
csv = '/content/gdrive/My Drive/ECG_data/test_1000.csv'
ECG_data = ECG_dataset(csv_file = csv,transform= transforms.ToTensor())

In [302]:
dataloader = DataLoader(ECG_data, batch_size=4,
                        shuffle=True, num_workers=4)
for i, batch in enumerate(dataloader):
    print(i, batch)

TypeError: ignored